# Assistente de Gerenciamento de Inventário" usando o LangGraph.
##  O script irá monitorar os níveis de inventário, prever faltas de estoque usando dados históricos e tendências, e automatizar processos de reabastecimento, enviando alertas ou fazendo pedidos quando necessário.



Parte 1: Instalação de Bibliotecas Necessárias
Primeiramente, precisamos instalar as bibliotecas que serão utilizadas no script. Execute a célula abaixo no seu Jupyter Notebook:

In [1]:
!pip install duckdb pandas matplotlib seaborn langgraph langchain-openai langchain-core openai python-dotenv graphviz


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.4 MB 7.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.4 MB 5.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/11.4 MB 5.9 MB/s eta 0:00:02
   --------------- ------------------------ 4.5/11.4 MB 6.4 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.4 MB 6.1 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.4 MB 6.3 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.4 MB 6.4 MB/s eta 0:00:01
   ----------------------------------- ---- 10.2/11.4 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 11.4/11.4 MB 6.3 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [duckdb]
   -------------------- ------------------- 1/2 [duckdb]
   -------------------- ------------------- 1/2

Parte 2: Importações e Configurações Iniciais

Nesta parte, importamos as bibliotecas necessárias e configuramos o ambiente.

In [ ]:
# Importações necessárias
import os
import duckdb  # Banco de dados embutido
import pandas as pd  # Manipulação de dados
import numpy as np  # Operações numéricas
import matplotlib.pyplot as plt  # Visualizações
import seaborn as sns  # Visualizações aprimoradas
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, START, END  # Para criar o fluxo de trabalho
from langchain_openai.chat_models import ChatOpenAI  # Modelo de linguagem
from langchain_core.prompts import ChatPromptTemplate  # Para criar prompts personalizados
from dotenv import load_dotenv  # Para carregar variáveis de ambiente

# Carregar variáveis de ambiente
load_dotenv()
os.environ["OPENAI_API_KEY"] = ''

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


Parte 3: Criação de um Banco de Dados Fictício com DuckDB
Vamos criar um banco de dados fictício que representa o inventário e histórico de vendas.

In [4]:
# Conectar ao banco de dados em memória
conn = duckdb.connect(database=':memory:')

# Criar dados fictícios de inventário
inventory_data = pd.DataFrame({
    'product_id': [f'P{str(i).zfill(3)}' for i in range(1, 21)],
    'product_name': [f'Produto {i}' for i in range(1, 21)],
    'current_stock': np.random.randint(0, 100, size=20),
    'reorder_level': np.random.randint(10, 50, size=20),
    'lead_time_days': np.random.randint(1, 15, size=20)
})

# Criar dados fictícios de vendas históricas
date_range = pd.date_range(end=pd.Timestamp.today(), periods=180)
sales_history = pd.DataFrame({
    'date': np.random.choice(date_range, size=500),
    'product_id': np.random.choice(inventory_data['product_id'], size=500),
    'quantity_sold': np.random.randint(1, 10, size=500)
})

# Inserir os dados no DuckDB
conn.execute("CREATE TABLE inventory AS SELECT * FROM inventory_data")
conn.execute("CREATE TABLE sales_history AS SELECT * FROM sales_history")


In [5]:
#testando o banco de dados
print(conn.execute("SELECT * FROM inventory").df())
print(conn.execute("SELECT * FROM sales_history").df())

   product_id product_name  current_stock  reorder_level  lead_time_days
0        P001    Produto 1              4             24               2
1        P002    Produto 2             39             13               1
2        P003    Produto 3             24             10               6
3        P004    Produto 4             79             44              11
4        P005    Produto 5              3             11               4
5        P006    Produto 6             47             10               9
6        P007    Produto 7             86             16              14
7        P008    Produto 8             32             43               6
8        P009    Produto 9             91             32               5
9        P010   Produto 10             47             42               4
10       P011   Produto 11             40             35               6
11       P012   Produto 12             60             38              10
12       P013   Produto 13             17          

Parte 4: Definição das Estruturas de Dados e Estado

Definimos as estruturas de dados que serão usadas no fluxo de trabalho.

In [6]:
# Estrutura do estado
class State(TypedDict):
    low_stock_products: pd.DataFrame
    demand_forecast: pd.DataFrame
    replenishment_orders: pd.DataFrame
    alerts: List[str]
    errors: List[str]


Parte 5: Implementação das Funções de Fluxo de Trabalho

5.1. Função para Identificar Produtos com Baixo Estoque

In [7]:
def check_inventory_levels(state: State) -> State:
    """
    Verifica o nível atual de estoque e identifica produtos abaixo do nível de reabastecimento.
    """
    try:
        # Consultar o banco de dados para obter o inventário atual
        inventory = conn.execute("SELECT * FROM inventory").df()
        
        # Identificar produtos com estoque abaixo do nível de reabastecimento
        low_stock = inventory[inventory['current_stock'] <= inventory['reorder_level']]
        
        # Atualizar o estado
        state['low_stock_products'] = low_stock
    except Exception as e:
        state['errors'].append(f"Erro ao verificar os níveis de inventário: {e}")
    return state


5.2. Função para Prever a Demanda Futura



In [8]:
def forecast_demand(state: State) -> State:
    """
    Prevê a demanda futura com base no histórico de vendas.
    """
    try:
        # Obter produtos com baixo estoque
        low_stock = state.get('low_stock_products')
        
        if low_stock is None or low_stock.empty:
            state['errors'].append("Nenhum produto com baixo estoque para prever demanda.")
            return state
        
        # Obter histórico de vendas
        sales = conn.execute("SELECT * FROM sales_history").df()
        
        # Combinar dados
        data = sales.merge(low_stock[['product_id']], on='product_id')
        
        # Prever demanda (neste exemplo simples, usaremos a média diária)
        forecast = data.groupby('product_id').agg({
            'quantity_sold': lambda x: x.sum() / 180  # Média diária nos últimos 180 dias
        }).reset_index()
        forecast.rename(columns={'quantity_sold': 'daily_average_sold'}, inplace=True)
        
        # Atualizar o estado
        state['demand_forecast'] = forecast
    except Exception as e:
        state['errors'].append(f"Erro ao prever a demanda: {e}")
    return state


5.3. Função para Gerar Ordens de Reabastecimento


In [10]:
def generate_replenishment_orders(state: State) -> State:
    """
    Gera ordens de reabastecimento com base na previsão de demanda e tempo de entrega.
    """
    try:
        low_stock = state.get('low_stock_products')
        forecast = state.get('demand_forecast')
        
        if low_stock is None or low_stock.empty or forecast is None or forecast.empty:
            state['errors'].append("Dados insuficientes para gerar ordens de reabastecimento.")
            return state
        
        # Combinar dados
        orders = low_stock.merge(forecast, on='product_id')
        
        # Calcular a quantidade necessária
        orders['replenishment_quantity'] = (
            orders['daily_average_sold'] * orders['lead_time_days']
        ).astype(int) - orders['current_stock']
        
        # Filtrar pedidos com quantidade positiva
        orders = orders[orders['replenishment_quantity'] > 0]
        
        # Atualizar o estado
        state['replenishment_orders'] = orders[['product_id', 'product_name', 'replenishment_quantity']]
    except Exception as e:
        state['errors'].append(f"Erro ao gerar ordens de reabastecimento: {e}")
    return state


5.4. Função para Enviar Alertas ou Fazer Pedidos


In [11]:
def send_alerts_or_place_orders(state: State) -> State:
    """
    Envia alertas ou faz pedidos de reabastecimento.
    """
    try:
        orders = state.get('replenishment_orders')
        
        if orders is None or orders.empty:
            state['alerts'].append("Nenhum produto necessita de reabastecimento imediato.")
            return state
        
        # Exemplo: Enviar um alerta para cada produto
        for _, row in orders.iterrows():
            alert = f"Produto {row['product_name']} (ID: {row['product_id']}) precisa ser reabastecido. Quantidade sugerida: {row['replenishment_quantity']} unidades."
            state['alerts'].append(alert)
            # Aqui, você poderia integrar com um sistema de pedidos automático
    except Exception as e:
        state['errors'].append(f"Erro ao enviar alertas ou fazer pedidos: {e}")
    return state


Parte 6: Construção do Fluxo de Trabalho com LangGraph

Vamos agora definir o fluxo de trabalho usando o LangGraph

In [12]:
# Inicializar o grafo de estado
workflow = StateGraph(State)

# Adicionar nós
workflow.add_node("check_inventory_levels", check_inventory_levels)
workflow.add_node("forecast_demand", forecast_demand)
workflow.add_node("generate_replenishment_orders", generate_replenishment_orders)
workflow.add_node("send_alerts_or_place_orders", send_alerts_or_place_orders)

# Definir as arestas
workflow.add_edge(START, "check_inventory_levels")
workflow.add_edge("check_inventory_levels", "forecast_demand")
workflow.add_edge("forecast_demand", "generate_replenishment_orders")
workflow.add_edge("generate_replenishment_orders", "send_alerts_or_place_orders")
workflow.add_edge("send_alerts_or_place_orders", END)

# Definir o ponto de entrada
workflow.set_entry_point("check_inventory_levels")

# Compilar o grafo
app = workflow.compile()


In [13]:
print(dir(workflow))


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_add_schema', '_all_edges', 'add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'branches', 'channels', 'compile', 'compiled', 'config_schema', 'edges', 'input_schema', 'managed', 'nodes', 'output_schema', 'schemas', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'state_schema', 'validate', 'waiting_edges']


Parte 87 Função Principal para Executar o Fluxo de Trabalho

Definimos uma função principal para executar o fluxo de trabalho.

In [14]:
def manage_inventory():
    """
    Executa o fluxo de gerenciamento de inventário.
    
    Returns:
        State: Estado final após o processamento.
    """
    initial_state = State(
        low_stock_products=None,
        demand_forecast=None,
        replenishment_orders=None,
        alerts=[],
        errors=[]
    )
    final_state = app.invoke(initial_state)
    return final_state


Parte 8: Executando o Script e Verificando os Resultados


In [15]:
# Executar o gerenciamento de inventário
final_state = manage_inventory()

In [16]:
# Verificar se houve erros
if final_state['errors']:
    print("Erros encontrados:")
    for error in final_state['errors']:
        print(f"- {error}")
else:
    # Exibir alertas
    if final_state['alerts']:
        print("Alertas:")
        for alert in final_state['alerts']:
            print(f"- {alert}")
    else:
        print("Nenhum alerta a ser exibido.")


Alertas:
- Nenhum produto necessita de reabastecimento imediato.


# Análise do Código de Gerenciamento de Inventário com LangGraph

## Conceitos Principais do LangGraph Utilizados

* **StateGraph**: 
  - Classe central do LangGraph que permite criar fluxos de trabalho baseados em estado
  - Gerencia a transição entre diferentes nós do grafo
  - Mantém o estado consistente durante toda a execução

* **State (TypedDict)**:
  - Estrutura de dados fortemente tipada que define o estado do workflow
  - Permite rastrear múltiplos aspectos do processo (inventário, previsões, alertas, etc.)

* **Nós do Grafo**:
  - Cada função representa um nó no grafo de workflow
  - Os nós processam e modificam o estado de forma sequencial
  - Retornam sempre um novo estado atualizado

* **START/END**:
  - Marcadores especiais do LangGraph que definem início e fim do workflow
  - Permitem estruturar o fluxo de execução de forma clara

## Análise do Código por Seções

### 1. Configuração Inicial
```python
from langgraph.graph import StateGraph, START, END
```
- Importa componentes essenciais do LangGraph
- Configura ambiente e dependências

### 2. Estrutura de Dados
```python
class State(TypedDict):
    low_stock_products: pd.DataFrame
    demand_forecast: pd.DataFrame
    replenishment_orders: pd.DataFrame
    alerts: List[str]
    errors: List[str]
```
- Define estrutura do estado usando TypedDict
- Cada campo representa um aspecto específico do processo

### 3. Funções de Processamento
```python
def check_inventory_levels(state: State) -> State:
    # ...
def forecast_demand(state: State) -> State:
    # ...
def generate_replenishment_orders(state: State) -> State:
    # ...
def send_alerts_or_place_orders(state: State) -> State:
    # ...
```
- Cada função é um nó do grafo
- Recebem e retornam o estado
- Seguem padrão de imutabilidade

### 4. Construção do Workflow
```python
workflow = StateGraph(State)
workflow.add_node("check_inventory_levels", check_inventory_levels)
# ...
workflow.add_edge(START, "check_inventory_levels")
# ...
app = workflow.compile()
```
- Cria grafo de estado
- Define nós e arestas
- Compila o workflow para execução

## Boas Práticas Observadas

* **Tratamento de Erros**:
  - Cada função possui try/catch
  - Erros são armazenados no estado
  - Permite rastreamento de problemas

* **Modularidade**:
  - Funções bem definidas com responsabilidade única
  - Facilita manutenção e testes

* **Tipagem**:
  - Uso de TypedDict para garantir consistência
  - Facilita detecção de erros em tempo de desenvolvimento

## Sugestões de Melhorias

* **Paralelização**:
  - Algumas operações poderiam ser paralelizadas
  - LangGraph suporta execução assíncrona

* **Validação de Estado**:
  - Adicionar validadores entre transições
  - Garantir integridade dos dados

* **Logging**:
  - Implementar sistema de logging mais robusto
  - Facilitar debug e monitoramento

* **Testes**:
  - Adicionar testes unitários para cada nó
  - Testar fluxo completo com diferentes cenários

Este código demonstra um uso prático e bem estruturado do LangGraph para criar um workflow de gerenciamento de inventário, seguindo boas práticas de programação e aproveitando os recursos da biblioteca.

Considerações Finais
Personalização:

Você pode ajustar os parâmetros de previsão de demanda para utilizar modelos estatísticos mais avançados ou integrar com bibliotecas como statsmodels ou scikit-learn.
Integre o sistema com APIs de fornecedores para automatizar os pedidos de reabastecimento.
Segurança e Validação:

Certifique-se de validar os dados de entrada e tratar exceções adequadamente para evitar falhas no sistema.
Escalabilidade:

Para sistemas maiores, considere o uso de bancos de dados mais robustos e a implementação de processamento assíncrono.